1. import package

In [1]:
# import cv2
# from PIL import Image

# image = cv2.imread('img/tryout.png', 1)
# edges = cv2.Canny(image, 500, 100)

# cv2.imshow("Edge Detection", edges)
# box = (250, 250, 750, 750)
# image = img.crop(box)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [2]:

# import numpy as np
# import cv2

# image = cv2.imread('img/tryout.png')
# y=50
# x=600
# w=500
# h=500
# crop = image[y:y+h, x:x+w]
# cv2.imshow('Image', crop)
# cv2.waitKey(0) 

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
def show(name, img, hold=True):
    cv2.imshow(name, img) 
    if hold :
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
def preprocessing(img, inv=True):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    __, binary = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
    if inv :
        binary = cv2.bitwise_not(binary)
    return binary
def contourOK(cc, min_w=30, min_h=30, min_area=300):
    x, y, w, h = cv2.boundingRect(cc)
    contour_area = cv2.contourArea(cc)
    area = w*h 
    if w < min_w or h < min_h :
        return False
    return contour_area > min_area 

def get_contours(img, min_w__=30, min_h__=30, min_area__=300):
    __ , contours, hierarchy  = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = [cc for cc in contours if contourOK(cc, min_w__, min_h__, min_area__) ]
    return contours

def crop_contour(img, contours):
    imgs = []
    for cc in contours: 
        x, y, w, h = cv2.boundingRect(cc)
        imgs.append(img[y:y+h, x:x+w])
    return imgs
def image_definition(ratio, orient):
    image_type = ''
    if ratio > 0.5 and orient == 'h':
        image_type = 'jawaban'
    elif ratio > 0.38 and orient == 'h':
        image_type = 'study_type'
    elif ratio > 0.21 and orient == 'h':
        image_type = 'header'
    elif ratio > 0.16 and orient == 'h':
        image_type = 'signature'
    elif ratio > 4.55 and orient == 'v':
        image_type = 'question_code'
    elif ratio > 2.0 and orient == 'v':
        image_type = 'birth_date'
    elif ratio > 1.3 and orient == 'v':
        image_type = 'name'
    elif ratio > 1.0 and orient == 'v':
        image_type = 'student_id'
        
    return image_type
def morph(img, inv = True, morph_type = 'ex', kernel_size=(1,10)):
    if inv :
        img = cv2.bitwise_not(img)
    
    if morph_type == 'ex' :
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    if morph_type == 'erode' :
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
        img = cv2.erode(img, kernel, iterations=1) 
        
    if morph_type == 'dilate' :
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
        img = cv2.dilate(img,kernel,iterations = 1)

    return img
 
img = cv2.imread("LJK-sample-001.jpg")

binary_inv = preprocessing(img.copy())

contours = get_contours(binary_inv)

imgs = crop_contour(img.copy(), contours)

for i, img_item in enumerate(imgs):
    h, w, c = img_item.shape
    if cv2.countNonZero(img_item[:,:,0]) > 0.2* h*w:
        ratio = h/w 
        orient = 'v' if h > w else 'h'
        window_name = "%s - LJK" % (image_definition(ratio, orient))

img_contour = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 1)

# -------------------------- MORPHOLOGICAL -------------------------------
binary = preprocessing(imgs[0].copy(), inv=False)

morph_img = morph(binary, True, 'dilate', (5, 4))
morph_img = morph(morph_img, False, 'erode', (14, 2))
morph_img = morph(morph_img, False, 'erode', (1, 4))

edged = cv2.Canny(morph_img, 90, 100)


contours = get_contours(edged, 20, 5, 10)
img_copy = imgs[0].copy()

boxs = []
for cc in contours:
    x, y, w, h = cv2.boundingRect(cc)
    cv2.rectangle(img_copy, (x-3,y-3), (x+w+3, y+h+3), (255, 0, 0), 1)
    boxs.append([x-3, y-3, x+w+3, y+h+3])
    
# --------------------------- Find Circle using Hough Circle -------------------
circles = cv2.HoughCircles(img_0, cv2.HOUGH_GRADIENT, 1, \
                           img_0.shape[0]/64, param1=200, param2=11, minRadius=6, maxRadius=10)

answer = {}
circle_group = []

mapping = {
    0 : 'A',
    1 : 'B',
    2 : 'C', 
    3 : 'D',
    4 : 'E',
    5 : '-'
}

# Draw detected circles
if circles is not None:
    circles = np.uint16(np.around(circles))

    
    for i in circles[0, :]:
        ymin, ymax, xmin, xmax = i[1] - i[2], i[1] + i[2], i[0] - i[2], i[0] + i[2]
        h, w = ymax - ymin, xmax - xmin
        
        center = [i[0], i[1]]
        box_id = [key for key, box in enumerate(boxs) if box[0] <= i[0] and \
                                                          box[2] >= i[0] and \
                                                          box[1] <= i[1] and \
                                                          box[3] >= i[1]][0]
        
        if cv2.countNonZero(img_0[ymin:ymax, xmin:xmax]) <= 0.55*h*w:
            cv2.circle(imgs[0], (i[0], i[1]), i[2], (0, 255, 0), 2)
            circle_group.append([box_id, center, 1])
        else :
            circle_group.append([box_id, center, 0])
    
    circle_group = sorted(circle_group, key=lambda x: x[1][0], reverse=False)
    circle_group = sorted(circle_group, key=lambda x: x[1][1], reverse=False)
    
    circle_idxs = list(np.array(circle_group)[:,0])
    circle_idxs_count = {i:circle_idxs.count(i) for i in circle_idxs}
    
    k = 1
    j = 0
    l = 1
    for key in circle_idxs_count.keys():
        circle_group_perkey = sorted([ item for item in circle_group if item[0] == key ], \
                                     key=lambda x: x[1][0], \
                                     reverse=False)
        
        list_answer = list(np.array(circle_group_perkey)[:, 2])
        answer[k] = mapping[list_answer.index(1) if 1 in list_answer else 5]
        
        if j < 4 :
            k = k + 20
            j += 1 
        else :
            j = 0
            l += 1 
            k = l

show("Jawaban", imgs[0])
for item in sorted(answer.items()):
    print("jawaban soal ke-%d : %s" % (item) )


AttributeError: 'NoneType' object has no attribute 'copy'